In [1]:
import numpy as np
import mujoco
import mediapy as media
import copy
import scipy
from robot_descriptions.skydio_x2_mj_description import MJCF_PATH

In [2]:
model = mujoco.MjModel.from_xml_path(MJCF_PATH)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model, height=480, width=640)

for act_id in range(4):
    model.actuator(act_id).ctrlrange = np.array([-1e4, 1e4])

model.opt.timestep = 1e-3

In [3]:
def compute_gains(configuration):
    # 1. Set configuration and find control that stabilizes it (ctrl0)
    newdata = mujoco.MjData(model)
    newdata = copy.copy(data)

    mujoco.mj_resetData(model, newdata)
    newdata.qpos = configuration
    # compute the control
    mujoco.mj_forward(model, newdata)
    newdata.qacc = 0
    mujoco.mj_inverse(model, newdata)

    # define control and configuration to linearize around
    ctrl0 = newdata.qfrc_inverse.copy() @ np.linalg.pinv(newdata.actuator_moment)
    qpos0 = newdata.qpos.copy()
    print(ctrl0, qpos0)

    # 2. Set weights
    R = np.eye(4) * 1e-3
    Q = np.block(
        [
            [np.eye(6), np.zeros((6, 6))],
            [np.zeros((6, 6)), np.eye(6)],
        ]
    )
    Q = np.eye(12)

    # 3. Compute LQR gains given weights
    mujoco.mj_resetData(model, newdata)
    newdata.ctrl = ctrl0
    newdata.qpos = qpos0

    # 4. Allocate the A and B matrices, compute them.
    A = np.zeros((2 * model.nv, 2 * model.nv))
    B = np.zeros((2 * model.nv, model.nu))
    epsilon = 1e-6
    flg_centered = True
    mujoco.mjd_transitionFD(model, newdata, epsilon, flg_centered, A, B, None, None)

    # Solve discrete Riccati equation.
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)

    # Compute the feedback gain matrix K.
    K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

    return ctrl0, K

In [4]:
# Parameters.
DURATION = 20  # seconds
FRAMERATE = 20  # Hz
BALANCE_STD = 0.3  # actuator units

dq = np.zeros(model.nv)

qpos0 = np.array([0, 0, 0.3, 1.0, 0, 0, 0])
zero_config = qpos0.copy()
ctrl0, K = compute_gains(zero_config)


# Reset data, set initial pose.
mujoco.mj_resetData(model, data)
data.qpos = qpos0

last_time = 0
phase = 0


frames = []
step = 0
while data.time < DURATION:
    # Get state difference dx.
    mujoco.mj_differentiatePos(model, dq, 1, zero_config, data.qpos)
    dx = np.hstack((dq, data.qvel)).T

    # LQR control law.
    data.ctrl = ctrl0 - K @ dx
    data.xfrc_applied[1, :3] = np.random.randn(3) * BALANCE_STD

    # Step the simulation.
    mujoco.mj_step(model, data)

    if data.time > phase * 2.0:
        phase += 1
        zero_config[phase % 3] = 0.3
        zero_config[(phase + 1) % 3] = 0
        zero_config[(phase + 2) % 3] = 0
        ctrl0, K = compute_gains(zero_config)

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]


[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.3 0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.3 0.  0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.3 0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.3 0.  0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.3 0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.3 0.  0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.3 0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]


This browser does not support the video tag.
